## 2.10 Tensorflow Datasets

In [1]:
import tensorflow as tf;
import sklearn.datasets;
import sklearn;
iris_ds = sklearn.datasets.load_iris(return_X_y=False)
import pandas as pd
import numpy as np
iris_data = pd.DataFrame(data=iris_ds.data,columns=iris_ds.feature_names)
from sklearn.model_selection import train_test_split
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
scaled_data = min_max_scaler.fit_transform(iris_data)
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(3)
label = encoder.fit_transform(iris_ds.target.reshape(-1,1))
label = label.todense()
trainx,testx,trainy,testy = train_test_split(scaled_data,label)

ModuleNotFoundError: No module named 'tensorflow'